Nim is two-player game starting with some number of piles of stones.
Each turn, the given player takes any (positive) number of stones from a chosen pile,
and the game is finished when no stones are left.

In normal play, the player who takes the last stone is the winner.

In 'misere' play, the player who takes the last stone is the loser.

Note:  Perfect play is not yet implemented for misere play.

In [1]:
import numpy as np
from copy import deepcopy

In [2]:
def argmax(a_dict, random = True):
    if len(a_dict) == 0:
        raise ValueError("Trying to find argmax of empty dict")
    max_value = max(a_dict.values())
    max_keys = [key for key in a_dict.keys() if a_dict[key] == max_value]
    if len(max_keys) >= 2 and random:
        return sample(max_keys)
    else:
        return max_keys[0]

def sample(A):
    return A[np.random.randint(len(A))]

In [3]:


class nim_state:
    def __init__(self,v, misere = False):        
        self.v = np.array(v).astype(int)
        self.num_piles = len(v)
        self.player = 0
        self.action_space = self.get_action_space()
        self.done = False
        self.misere = misere
        
    def get_action_space(self):
        actions = []
        for j in range(self.num_piles):
            for i in range(int(self.v[j])):
                action = np.zeros(self.num_piles)
                action[j] = i - self.v[j]
                actions.append(action)
        return actions
    
    def get_action_from_int(self, i):
        partial_sum = 0
        for j in range(len(self.v)):
            if i < partial_sum + self.v[j]:
                    action = np.zeros(self.num_piles)
                    action[j] =  i - partial_sum - self.v[j]
                    return action
            else:
                partial_sum += self.v[j]
        else:
            raise ValueError("This action not allowed")
            
#     def simulate_step(self, i):
#         #returns next state without updating current state
#         action = self.get_action_from_int(i)
#         next_v = self.v + action
#         if any(next_v < 0):
#             raise ValueError("This action not allowed")
#         else:
#             next_state = nim_state(next_v.astype(int))
#             next_state.player = 1 - self.player
#             return next_state
        
    
    def take_action(self, action, inplace = True):
        next_v = self.v + action
        if any(next_v < 0):
            raise ValueError("This action not allowed")
        else:
            if inplace:
                self.v = next_v.astype(int)
                self.player =  1 - self.player
            else:
                new_state = deepcopy(self)
                new_state.v = next_v.astype(int)
                new_state.player =  1 - self.player
                return new_state
                
            #self.action_space = self.get_action_from_int()
            
    def is_done(self):
        return all(self.v == np.zeros(self.num_piles))
    
    def reward(self):
        done = self.is_done()
        if done and self.player == 0:
            reward = 0 #player 1 win
        elif done and self.player == 1:
            reward = 1  #player 0 win
        else:
            reward = 0
        return reward

    def winner(self):
        done = self.is_done()
        if done and self.player == 0:
            winner = 1
        elif done and self.player == 1:
            winner = 0
        else:
            winner = None
            
        if self.misere:
            return 1 - winner
        return winner

    
            
    def step(self, i, inplace = True):
        #self.take_action(self.action_space[i])
        if inplace:
            self.take_action(self.get_action_from_int(i), inplace = True)
            observation = self.v
            done = all(self.v == np.zeros(self.num_piles))
            self.done = done


            #self.reward = reward
            reward = self.reward()
            info = None
            return observation, reward, done, info
        else:
            new_state = self.take_action(self.get_action_from_int(i), inplace = False)
            observation = new_state.v
            done = all(new_state.v == np.zeros(new_state.num_piles))
            new_state.done = done


            #self.reward = reward
            reward = new_state.reward()
            info = None
            return observation, reward, done, info, new_state
    
    def perfect_action(self):
        self.action_space = self.get_action_space()
        for i in range(len(self.action_space)):
            if reduce(lambda x,y: x ^ y, [int(coord) for coord in self.v + self.action_space[i]] )  == 0:
                return i
        else:
            return None
        
    def random_action(self):
        return np.random.randint(sum(self.v))
    
    def num_actions(self):
        return sum(self.v)
    
    def perfect_play_winner(self):
        current_state_is_won = int(reduce(lambda x,y: x ^ y, game.v)  == 0)
        if self.player == 0:
            return current_state_is_won
        elif self.player == 1:
            return 1 - current_state_is_won

Nim is a *solved game*.  That is, the strategy for optimal play is known and easily found at every step.  Here is a game simulated for optimal play.

In [4]:
v = [1000,1000,1000]
print 'Winner with perfect play: Player', int(reduce(lambda x,y: x ^ y, v)  == 0)
env = nim_state(v)
observation = env.v
print 'state:', observation, 'to play: %d' % env.player
for i in range(sum(v)):
    action = env.perfect_action()
    if action is None:
        action = env.random_action()
    observation, reward, done, info = env.step(action)
    print 'state:', observation, 'to play: %d' % env.player
    if done:
        print 'Player %d wins!' % env.winner()
        break

Winner with perfect play: Player 0
state: [1000 1000 1000] to play: 0
state: [   0 1000 1000] to play: 1
state: [   0  465 1000] to play: 0
state: [  0 465 465] to play: 1
state: [  0 465   6] to play: 0
state: [0 6 6] to play: 1
state: [0 6 1] to play: 0
state: [0 1 1] to play: 1
state: [0 1 0] to play: 0
state: [0 0 0] to play: 1
Player 0 wins!


In [5]:
class MonteCarloTree:
    def __init__(self, game_state, budget = 100, c = 1, twoplayer = True, log_tree_building = False):
        
        self.root = Node(game_state, index = 0)
        self.budget = budget
        self.tree = {0:self.root}  #tree is a dictionary of Nodes
        self.max_steps = 100
        self.c = 1
        self.twoplayer = twoplayer
        self.log_tree_building = log_tree_building
        

    def UCTSearch(self):      
        nsteps = 1
        while nsteps < self.budget:
            nsteps += 1
            next_node = self.TreePolicy()  #creates a new node and adds to tree

            
            if self.root.is_complete and self.root.is_won:
                #if self.log_tree_building:
                print "Found winning move."
                break
            elif self.root.is_complete and not self.root.is_won:
                #if self.log_tree_building:
                print "Guaranteed loss! Returning None."
                return None
            else:
                Q = self.Simulate(next_node)
                self.BackPropogate(next_node, Q)
        return self.BestChild(self.root, explore = False).action
        
    def TreePolicy(self):
        node = self.root
        while len(node.children) > 0:  #explore until a leaf is found
            if len(node.children) < node.state.num_actions(): #if not fully expanded
                return self.Expand(node)
            else:
                node = self.BestChild(node, self.c)
        return self.Expand(node)
                
    def Simulate(self,node):
        
        if node.state.is_done():
#             if node.state.reward() > 0:  ### assuming for now reward >0 means 'won game'
#                 reward = np.inf
#             else:
#                 reward = node.state.reward()
            reward = node.state.reward()
            if reward is None:
                raise ValueError('eh?', node.state.v)
            return reward
        else:
            nsteps = 0
            current_state = deepcopy(node.state)
            #Should we be updating the tree here?
            while nsteps < self.max_steps:
                action = current_state.random_action()
                observation, reward, done, info = current_state.step(action)
                if done:
                    return reward
            else:
                raise ValueError("Max number of steps exceeded")
    
    def Expand(self,node):
        if len(node.children) >= node.state.num_actions():
            raise ValueError("Node fully expanded already")
        else:
            action = len(node.children)
            new_index = max(self.tree.keys()) + 1
            if self.log_tree_building:
                print "Adding node %d from parent %d with action %d" % (new_index, node.index, action)
            observation, reward, done, info, new_state = node.state.step(action, inplace = False)
            new_child = Node(new_state, new_index, action = action, parent = node.index)
            node.children.append(new_index)
            self.tree[new_index] = new_child
              
            self.BackPropogate_EndGame(new_index)
                            
            return new_child
            
            
    def BackPropogate_EndGame(self, index):
        node = self.tree[index]
        
        if self.twoplayer:
            if node.state.done:
                node.is_complete = True
                if node.state.winner == node.state.player:
                    node.is_won = True
                else:
                    node.is_won = False
                

            
            elif any([self.tree[child_index].is_complete and not self.tree[child_index].is_won for child_index in node.children]):
                #if any child is a loss to that player, the current node is a WIN for the current player.
                self.tree[index].is_complete = True
                self.tree[index].is_won = True

            elif len(node.children) >= node.state.num_actions(): #fully expanded
                if all([self.tree[child_index].is_complete and self.tree[child_index].is_won for child_index in node.children]):
                    self.tree[index].is_complete = True
                    self.tree[index].is_won = False
        else:
            #single-player game
            if node.state.done:
                node.is_complete = True
                if node.state.winner == node.state.player:
                    node.is_won = True
                else:
                    node.is_won = False
            elif any([self.tree[child_index].is_complete and not self.tree[child_index].is_won for child_index in node.children]):
                #if any child is a win, the current node is a win for the current player.
                self.tree[index].is_complete = True
                self.tree[index].is_won = True
            elif len(node.children) >= node.state.num_actions(): #fully expanded
                if all([self.tree[child_index].is_complete and not self.tree[child_index].is_won for child_index in node.children]):
                    #if ALL children are losses the the current node is a loss.                    
                    self.tree[index].is_complete = True
                    self.tree[index].is_won = False
                    
        
        if node.is_complete:
            if node.is_won:
                end_string = 'win'
            else:
                end_string = 'loss'
            if self.log_tree_building:
                print "Node %d is complete and is a %s!!" % (node.index, end_string)
            
            if not node.parent is None:
                self.BackPropogate_EndGame(node.parent)


    
    def BackPropogate(self, node, reward):
        node_index = node.index
        while not node_index is None:
            node = self.tree[node_index]
            node.Q += reward
            node.N += 1
            if self.twoplayer:
                reward = -reward
            node_index = node.parent
    
    def BestChild(self, node, explore = True):
        #Perform Sophie's Choice
        if len(node.children) == 0:
            raise ValueError("Node has no children")
        if explore and node.is_complete:
            raise ValueError("Node has already been fully explored")
        
        S = {}
        
        for i in node.children:
            child = self.tree[i]
            if self.twoplayer:
                if child.is_complete and child.is_won:
                    #this child is a loss to the current node's player.  No need to investigate further.
                    continue
                if child.is_complete and child.is_won == False:
                    #this child is a WIN to the current node's player.  Take this move and don't explore further.
                    return child
            else:
                raise ValueError("not implemented")


            if explore:

                    if child.N == 0:
                        S[i] = np.inf
                    else:
                        S[i] = child.Q / child.N + self.c * np.sqrt(  2 * np.log( node.N ) / child.N  )

            else:
                try:
                    S[i] = child.Q / child.N
                except:
                    raise ValueError("Node %d has N = 0" % i)
        child = self.tree[argmax(S)]  #should we return an INDEX?
        return child

class Node:
    def __init__(self, state, index, action = None, parent = None):
        self.state = state
        self.action = action
        self.children = []
        self.index = index
        self.parent = parent
        self.Q = 0  #total reward
        self.N = 0  #total visit count
        self.is_complete = False # Set to True if subtree with this root is FULLY expanded to endgame
        self.is_won = None #Set to True if game is a win for the CURRENT player, 
                           #False if game is a loss for the CURRENT player, otherwise None

In [12]:
v = [5,5]

misere = False

game = nim_state(v, misere = misere)
print 'Player %d now choosing action from state %s.  Perfect play winner: %d.' % (game.player, game.v, game.perfect_play_winner())    
for i in range(100):
    game_tree = MonteCarloTree( nim_state(game.v, misere = misere), budget = 100)
    action = game_tree.UCTSearch()
    print 'Size of tree: %d' % len(game_tree.tree)

    if action is None:
        print "No good move.  Taking random action."
        action = game.random_action()
    (observation, reward, done, info) = game.step(action)
    print ''
    print 'Player %d now choosing action from state %s.  Perfect play winner: %d.' % (game.player, game.v, game.perfect_play_winner())    
    
    if done:
        print 'Player %d wins!' % game.winner()
        break


Player 0 now choosing action from state [5 5].  Perfect play winner: 1.
Size of tree: 100

Player 1 now choosing action from state [5 2].  Perfect play winner: 1.
Size of tree: 100

Player 0 now choosing action from state [2 2].  Perfect play winner: 1.
Guaranteed loss! Returning None.
Size of tree: 25
No good move.  Taking random action.

Player 1 now choosing action from state [1 2].  Perfect play winner: 1.
Found winning move.
Size of tree: 10

Player 0 now choosing action from state [1 1].  Perfect play winner: 1.
Guaranteed loss! Returning None.
Size of tree: 5
No good move.  Taking random action.

Player 1 now choosing action from state [0 1].  Perfect play winner: 1.
Found winning move.
Size of tree: 2

Player 0 now choosing action from state [0 0].  Perfect play winner: 1.
Player 1 wins!
